In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [46]:
data = pd.read_csv("./data/2015_사업보고서_01_재무상태표_연결_20200515.txt", sep = "\t", encoding = "cp949")
df = data.copy()
df2016 = pd.read_csv("./data/2016_사업보고서_01_재무상태표_연결_20200626.txt", sep = "\t", encoding = "cp949")
df2017 = pd.read_csv("./data/2017_사업보고서_01_재무상태표_연결_20200626.txt", sep = "\t", encoding = "cp949")
df2018 = pd.read_csv("./data/2018_사업보고서_01_재무상태표_연결_20200627.txt", sep = "\t", encoding = "cp949")
df2019 = pd.read_csv("./data/2019_사업보고서_01_재무상태표_연결_20200623.txt", sep = "\t", encoding = "cp949")

In [ ]:
print(df.isnull().sum())
# 결측치 처리 필요!
print(df.info())

In [ ]:
# 변수 
# 재무제표종류 종목코드 회사명 시장구분 업종 업종명 결산월 결산기준일 
# 보고서종류 통화 항목코드 항목명 당기 전기 전전기 Unnamed: 15
print(" ".join(df.columns))
print(" ".join(df2016.columns))
print(" ".join(df2017.columns))
print(" ".join(df2018.columns))
print(" ".join(df2019.columns))

In [ ]:
# 재무제표종류
print(df["재무제표종류"].unique())
print(df2016["재무제표종류"].unique())
print(df2017["재무제표종류"].unique())
print(df2018["재무제표종류"].unique())
print(df2019["재무제표종류"].unique())
print(df["재무제표종류"].value_counts())

In [ ]:
# 종목코드 : 한국거래소에서 관리하는 회사코드로 6자리로 구성
def unique(data):
    print(len(data["종목코드"].unique()))

unique(df)
unique(df2016)
unique(df2017)
unique(df2018)
unique(df2019)

In [ ]:
# 회사명 : 제출인의 회사명칭
print(len(df["회사명"].unique()))
print(len(df2016["회사명"].unique()))
print(len(df2017["회사명"].unique()))
print(len(df2018["회사명"].unique()))
print(len(df2019["회사명"].unique()))

In [8]:
# 시장구분 : 유가증권시장상장법인, 코스닥시장상장법인(Kosdaq)
# 유가증권시장상장법인 : Kospi
# 코스닥 시장은 IT(Information technology), BT(Bio technology), CT(Culture technology) 
# 기업과 벤처기업의 자금조달을 목적으로 1996년 7월 개설된 첨단 벤처기업 중심 시장입니다.
print(df["시장구분"].unique())

['코스닥시장상장법인' '유가증권시장상장법인']


In [ ]:
# 업종, 업종명
# 통계청 통계분류 포털 : https://kssc.kostat.go.kr:8443/ksscNew_web/link.do?gubun=001
print(df["업종명"].value_counts())
company_name = df["업종명"].unique()
company_name_sum = df["업종명"].value_counts().sort_values()
df_company_name = pd.DataFrame({"업종명":company_name, "합계":company_name_sum})

# 업종
print(df["업종"].value_counts())
company = df["업종"].unique()
company_sum = df["업종"].value_counts().sort_values()
df_company = pd.DataFrame({"업종":company, "합계":company_sum})

# 업종과 업종명이 다른 행 추출
com_left = pd.merge(df_company_name, df_company, how = "left")
print(com_left[com_left["업종"].isnull()])

com_right = pd.merge(df_company_name, df_company, how = "right")
print(com_right[com_right["업종명"].isnull()])

# 업종 != 업종명 처리방법 회의 필요

In [ ]:
# 결산월, 결산기준일
print(df["결산월"].unique())
print(df["결산기준일"].unique())

In [ ]:
# data
def str_to_float(data):
    import numpy as np
    
    data["당기"] = data["당기"].str.replace(",", "")
    data["전기"] = data["전기"].str.replace(",", "")
    data["전전기"] = data["전전기"].str.replace(",", "")
    data[["당기", "전기", "전전기"]] = data[["당기", "전기", "전전기"]].astype(np.float32)
    return data

str_to_float(df)
str_to_float(df2016)
str_to_float(df2017)
str_to_float(df2018)
str_to_float(df2019)

In [ ]:
# 필요 없는 변수 제거
def delete_col(data):
    data.drop("통화", axis = 1, inplace = True)
    data.drop("Unnamed: 15", axis = 1, inplace = True)
    return data

delete_col(df)
delete_col(df2016)
delete_col(df2017)
delete_col(df2018)
delete_col(df2019)

In [293]:
# 회사명 
company = list(df["회사명"].unique())

col_num = []
col_num_name = []
for idx, name in enumerate(company):
    a = df[df["회사명"] == name][["항목명", "당기"]].T
    a.rename(columns = a.iloc[0,:], inplace = True)
    a.drop(index = "항목명", inplace = True)
    col_num.append(len(a.columns))
    col_num_name.append(name)
    
    
# 가장 많은 항목명을 가진 회사
for i in range(len(col_num)):
    if col_num[i] == max(col_num):
        print(col_num_name[i])

농우바이오


In [294]:
a = df[df["회사명"] == "농우바이오"][["항목명", "당기"]].T
a.rename(columns = a.iloc[0,:], inplace = True)
a.drop(index = "항목명", inplace = True)
print(a)

   자산 [abstract]         유동자산     현금및현금성자산        현금성자산       매도가능금융자산  \
당기           NaN  1.10246e+11  1.28772e+10  1.28772e+10    1.81281e+08   

            매도가능금융자산       만기보유금융자산          만기보유금융자산 매출채권 및 기타유동채권  \
당기       1.81281e+08      8.084e+07         8.084e+07   3.00516e+10   

           매출채권  ...       해외사업환산손익       매도가능금융자산평가이익       지분변동차액  \
당기  3.04912e+10  ...    5.43001e+09        4.09875e+07 -7.18345e+06   

      이익잉여금(결 손 금)       법정적립금        임의적립금       미처분이익잉여금(미처리결손금)  \
당기     1.35262e+11  1.2726e+10  6.70326e+10            5.55032e+10   

          비지배지분         자본총계      자본과부채총계  
당기  7.67658e+07  1.80592e+11  2.16354e+11  

[1 rows x 228 columns]


### 항목명 처리

In [265]:
def duplication_check(data):
    # 항목코드 중복 확인
    lst_entity_코드 = []
    lst_entity_명 = []
    lst_without_entity_코드 = []
    lst_without_entity_명 = []

    # 각 리스트에 코드 및 코드명 append
    for idx in range(len(data)):
        if "entity" in data["항목코드"][idx]:
            lst_entity_코드.append(data["항목코드"][idx].lower())
            lst_entity_명.append(data["항목명"][idx].lower())
        else:
            lst_without_entity_코드.append(data["항목코드"][idx].lower())
            lst_without_entity_명.append(data["항목명"][idx].lower())

    # !entity 코드 딕셔너리
    dic_without_entity = {lst_without_entity_코드[idx]:[] for idx in range(len(lst_without_entity_코드))}

    for i, j in zip(lst_without_entity_코드, lst_without_entity_명):
        dic_without_entity[i].append(j)

    check_duplication = {i:len(set(j)) for i, j in dic_without_entity.items()}
    
    return lst_entity_코드, lst_entity_명, lst_without_entity_코드, lst_without_entity_명, dic_without_entity, check_duplication

lst_entity_코드, lst_entity_명, lst_without_entity_코드, lst_without_entity_명, dic_without_entity, check_duplication = duplication_check(df)

In [266]:
dic_entity = {lst_entity_코드[idx]:[] for idx in range(len(lst_entity_코드))}

for i, j in zip(lst_entity_코드, lst_entity_명):
    dic_entity[i].append(j)

check_duplication_entity = {i:len(set(j)) for i, j in dic_entity.items()}

dic_entity_명_코드 = {lst_entity_명[idx]:[] for idx in range(len(lst_entity_명))}

for i, j in zip(lst_entity_명, lst_entity_코드):
    dic_entity_명_코드[i].append(j)
    
dic_entity_명_코드

In [138]:
def extract_element_id(data):
    element_idx_lst_entity = []
    element_idx_lst_without_entity = []
    
    for idx in range(len(data)):
        if "entity" in data["항목코드"][idx]:
            element_idx_lst_entity.append(data["항목코드"][idx])
        else:
            element_idx_lst_without_entity.append(data["항목코드"][idx])
            
    entity = sorted(set(element_idx_lst_entity), reverse = True)
    without_entity = sorted(set(element_idx_lst_without_entity), reverse = True)
    
    return entity, without_entity

a, b = extract_element_id(df2019)
pd.Series(b).to_csv("./2019항목명.csv", encoding = "euc-kr", index = False)

### 재무제표 양식 기반 entity 없는 항목명 처리

In [300]:
elementid = pd.read_csv("./data/재무제표양식.csv", encoding = "utf-8")
# '한글 Label', 'Element ID'
df_element = elementid[['한글 Label', 'Element ID']]
df_element = df_element[df_element["Element ID"].notnull()]

df_element.columns = df_element.columns.str.replace(" ", "_")
lst_element = df_element["Element_ID"].unique()

re_lst_element = []
for i in lst_element:
    a = re.findall("_\w{1,}", i)[0].lower()
    re_lst_element.append(a)
 
dic_element = {i:[] for i in re_lst_element}
for i, j in zip(re_lst_element, df_element["한글_Label"]):
    dic_element[i].append(j)
    
    
error_idx = []
for idx in range(len(df)):
    try:        
        if "entity" not in df["항목코드"][idx]:
            df["항목명"][idx] = dic_element[re.findall("_\w{1,}", df["항목코드"][idx])[0].lower()][0]
    except Exception as e:
        error_idx.append(idx)

C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [303]:
df.iloc[error_idx, :].to_csv("./2015에러행.csv", encoding = "euc-kr", index = False)

In [304]:
len(df)

64605